<a href="https://colab.research.google.com/github/emoceanographer/WWF/blob/master/FAO_commodity_shifts_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import pandas as pd
from scipy import stats
import numpy as np


In [0]:
def unique(long_list):
  unique_list = []
  for element in long_list: 
    if element not in unique_list:
      unique_list.append(element)
  return unique_list

def max_change(value,year):
  max_value = max(value)
  max_index = value.index(max_value)
  return([max_value, year[max_index]])

def change_calc(value,year_offset):
  """Calculates the difference between elements 'year_offset' apart within a list"""
  change_value = []
  for i in range(year_offset,len(value)): 
    change_value.append(value[i] - value[i-year_offset])
  return change_value

def fv_mean(listA):
  output = []
  listA = list(listA)
  for item in listA:
    if len(item)>0:
      output.append(item[0])
  if len(output)>0:
    mean_output = stat.mean(output)
  else:
    mean_output = output
  return mean_output

In [0]:
def country_all_analysis(country_df, domain):
  """Country data frame is expecting an FAO data set for a country; 'domain'
  should be the "element" we are interested in"""
  
  if country_df['Element'].str.contains(domain).any():
    data = country_df[country_df['Element'].str.contains(domain)]

    slope,t,t,t,t = stats.linregress(data['Year'], data['Value'])
    value = list(data['Value'])

    change_1yr = change_calc(value,1)
    max_1yr = max_change(change_1yr, list(data['Year'][1:]))
    change_5yr = change_calc(value,5) 
    max_5yr = max_change(change_5yr, list(data['Year'][5:]))

    output = {'Slope'+'_'+domain: slope, 'Delta1yr'+'_'+domain: max_1yr[0],
              'D1y_yr': max_1yr[1], 'Delta5yr'+'_'+domain: max_5yr[0],
              'D5y_yr': max_5yr[1]
             }
  else:
    output = {'Slope'+'_'+domain: float('nan'), 'Delta1yr'+'_'+domain: float('nan'),
              'D1y_yr': float('nan'), 'Delta5yr'+'_'+domain: float('nan'),
              'D5y_yr': float('nan')
             }
  return output

In [0]:
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
rainbow = cm.get_cmap('rainbow',12)

def plot_color(x,y,t,x_label,y_label,title):
  """Plots x vs y with colors set by t; labels also entered as inputs"""
  fig = plt.figure(figsize = (8,8))
  ax = fig.add_subplot(1,1,1) 
  ax.set_xlabel(x_label, fontsize = 15)
  ax.set_ylabel(y_label, fontsize = 15) # grab units from sheet
  ax.set_title(title, fontsize = 20) 
  # grab the country name and commodity from the data
  
  #rainbow = cm.get_cmap('rainbow',len(t))
  #for i in range(0,len(x)): # goes through each element
   # ax.scatter(x[i], y[i], c = rainbow[i])
  plt.scatter(x, y, c = t, cmap = 'rainbow')
  plt.show()
  
def plot_simple(x,y,x_label, y_label, title): 
  fig = plt.figure(figsize = (4,4))
  ax = fig.add_subplot(1,1,1) 
  ax.set_xlabel(x_label, fontsize = 10)
  ax.set_ylabel(y_label, fontsize = 10) # grab units from sheet
  ax.set_title(title, fontsize = 10) 
  
  plt.scatter(x,y)
  plt.show()
  
# plots, if desired
def ctry_plot(country,df,df_gdp,commodity):
  country_ID = df['Country'].str.contains(country)
  country_data = df[country_ID] # this seems to grab only the "true" values from the above
  commodity_data = country_data[country_data['Item'].str.contains(commodity)]

  country_ID = df_gdp['Area'].str.contains(country) # grabs same country from GDP data
  country_data = df_gdp[country_ID]
  gdp_data = country_data[country_data['Item'].str.contains(gdp_type)]

  list1 = [] # commodity change
  list2 = [] # GDP change
  list3 = [] # year
  
  for year in unique(list(commodity_data['Year'])):

    if gdp_data['Year'].astype(str).str.contains(str(year)).any(): # if that year exists
      yr_data1 = commodity_data[commodity_data['Year'].astype(str).str.contains(str(year))]
      yr_data2 = gdp_data[gdp_data['Year'].astype(str).str.contains(str(year))]
      list1.append(yr_data1['Value'].values[0])
      list2.append(yr_data2['Value'].values[0])
      list3.append(year)

  if list1:
    slope,intercept,r_value,p_value,t = stats.linregress(list2,list1)

    plot_color(list2,list1,list3,'GDP per capita','meat supply (kg/capita/year)',country)
    plot_simple(list3,list1, 'time', 'protein supply',country)
    plot_simple(list3,list2, 'time', 'GDP',country)

In [0]:
def slopes_counter(slopes,pvals):
  """Treats NaN entries as nonexistant; proportion is from those with data"""
  length = sum(1 for x in slopes if not np.isnan(x))
  prop_pos = sum(1 for x in slopes if x > 0) / length # gets proportion
  counter = 0
  for i in range(0,len(slopes)):
    if slopes[i] > 0 and pvals[i]<0.05:
      counter = counter + 1

  prop_pos_sig = counter / length
  return prop_pos, prop_pos_sig

In [0]:
def country_slope_calcs(commodity, GDPtype,yrstart):
  """for a particular commodity as defined in FAO's data, goes through each country
  and regresses the GDP (either full or per capita) versus the protein supply quantity 
  of that commodity """
  
  country_dict = {}
  country_dict = {}

  commodity1 = commodity
  commodity2 = GDPtype

  for country in country_list: 
    country_ID = df['Country'].str.contains(country)
    country_data = df[country_ID] # this seems to grab only the "true" values from the above
    commodity_data = country_data[country_data['Item'].str.contains(commodity1)]

    country_ID = df_gdp['Area'].str.contains(country) # grabs same country from GDP data
    country_data = df_gdp[country_ID]
    gdp_data = country_data[country_data['Item'].str.contains(commodity2)]

    list1 = [] # commodity change
    list2 = [] # GDP change
    list3 = [] # year
    
    yearlist = unique(list(commodity_data['Year']))
    yearlist = [i for i in yearlist if i>yrstart] # selects only years larger than threshold
    for year in yearlist:

      if gdp_data['Year'].astype(str).str.contains(str(year)).any(): # if that year exists
        yr_data1 = commodity_data[commodity_data['Year'].astype(str).str.contains(str(year))]
        yr_data2 = gdp_data[gdp_data['Year'].astype(str).str.contains(str(year))]
        list1.append(yr_data1['Value'].values[0])
        list2.append(yr_data2['Value'].values[0])
        list3.append(year)

    if list1:
      slope,intercept,r_value,p_value,t = stats.linregress(list2,list1)
      country_dict[country] = {'Slope': slope, 'Intercept': intercept, 'r^2': r_value, 'p': p_value}

    else:
      country_dict[country] = {'Slope': float('nan'), 'Intercept': float('nan'), 'r^2': float('nan'), 'p': float('nan')}
  return country_dict

In [0]:
# Choose a data source
path_other = "gdrive/My Drive/Colab Notebooks/FAOSTAT_data_6-18-2019.csv" # milk, poultry
path_other = "gdrive/My Drive/Colab Notebooks/faostat_chixmilk_pc.csv" # milk, poultry per capita
path_meattype = "gdrive/My Drive/Colab Notebooks/faostat_meattype_kg.csv" # meat by type
path_meatall = "gdrive/My Drive/Colab Notebooks/faostat_meat_kg.csv" # meat total
path_gdp = "gdrive/My Drive/Colab Notebooks/FAOSTAT_data_gdp.csv" # fao gdp data over time
path_countrycats = "gdrive/My Drive/Colab Notebooks/country_classes.csv"

path_fish = 'gdrive/My Drive/Colab Notebooks/TS_FI_AQUACULTURE.csv' # FAO fish stat date
path_fish_IDS = 'gdrive/My Drive/Colab Notebooks/CL_FI_SPECIES_GROUPS.csv' # species codes data
path_fish_country = 'gdrive/My Drive/Colab Notebooks/aqua_country_codes.csv' # country codes

In [0]:
yearstart = 1950 # choose when to start analysis

# Run all meat version

In [0]:
# Creates a data frame from the data in 'path' and extracts the countries of interest
df = pd.read_csv(path_meatall)
df_gdp = pd.read_csv(path_gdp) # loads gdp data
country_list = unique(list(df['Country']))

In [0]:
# Runs the analysis and saves the meat version
commodity = 'Meat'
gdp_type = 'Gross Domestic Product per capita'
country_dict = country_slope_calcs(commodity, gdp_type ,yearstart) # capitalization matters!
countrydf = pd.DataFrame.from_dict(country_dict, orient = 'index')
savepath = 'gdrive/My Drive/Colab Notebooks/' + commodity + '_GDPpc_slopes.csv' # hopefully will always save 
# to the correct commodity
countrydf.to_csv(savepath)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


In [0]:
# plots by country 
ctry_plot('Belgium',df,df_gdp,commodity)

In [0]:
# Let's run some stats on these GDP / consumption relationships!
# Proportion of countries with positive slope:

slopes = countrydf.Slope.astype(float) # makes numbers not strings
pvals = countrydf.p.astype(float)
[pp,sig] = slopes_counter(slopes,pvals)

print(commodity)
print('Proportion positive:', pp)
print('Proportion positive and significant:', sig)

Meat
Proportion positive: 0.8
Proportion positive and significant: 0.6971428571428572


# Run by commodity version

In [0]:
# Creates a data frame from the data in 'path' and extracts the countries of interest
df = pd.read_csv(path_meattype)
df_gdp = pd.read_csv(path_gdp) # loads gdp data
country_list = unique(list(df['Country']))

In [0]:
# Runs the analysis for each commodity
commodity_list = unique(list(df['Item']))

for commodity in commodity_list:
  country_dict = country_slope_calcs(commodity, 'Gross Domestic Product per capita', yearstart) # capitalization matters!
  countrydf = pd.DataFrame.from_dict(country_dict, orient = 'index')
  savepath = 'gdrive/My Drive/Colab Notebooks/' + commodity + '_GDPpc2002_slopes.csv' # hopefully will always save 
    # to the correct commodity
  countrydf.to_csv(savepath)
  
  slopes = countrydf.Slope.astype(float) # makes numbers not strings
  pvals = countrydf.p.astype(float)
  [pp,sig] = slopes_counter(slopes,pvals)

  print(commodity)
  print('Proportion positive:', pp)
  print('Proportion positive and significant:', sig)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


Bovine Meat
Proportion positive: 0.5263157894736842
Proportion positive and significant: 0.25146198830409355
Mutton & Goat Meat
Proportion positive: 0.6140350877192983
Proportion positive and significant: 0.29239766081871343
Poultry Meat
Proportion positive: 0.7543859649122807
Proportion positive and significant: 0.4853801169590643
Meat, Other
Proportion positive: 0.52046783625731
Proportion positive and significant: 0.21052631578947367
Pigmeat
Proportion positive: 0.6204819277108434
Proportion positive and significant: 0.3493975903614458


# Country Grouping by Income Group

In [0]:
## Group countries in regions and calculate aggregate statistics
categorydf = pd.read_csv(path_countrycats)

In [0]:
# MANUALLY CODED FOR NOW
meat_path = 'gdrive/My Drive/Colab Notebooks/Meat_GDPpc2002_slopes.csv'
bovine_path = 'gdrive/My Drive/Colab Notebooks/Bovine Meat_GDPpc2002_slopes.csv'
mutton_path = 'gdrive/My Drive/Colab Notebooks/Mutton & Goat Meat_GDPpc2002_slopes.csv'
poultry_path = 'gdrive/My Drive/Colab Notebooks/Poultry Meat_GDPpc2002_slopes.csv'
pig_path = 'gdrive/My Drive/Colab Notebooks/Pigmeat_GDPpc2002_slopes.csv'

In [0]:
income_groups = unique(categorydf['Income group'])
income_groups.remove('x') # removes a spurious 'x' category

In [0]:
import csv
path_list = {'meat': meat_path, 'bovine': bovine_path, 'mutton': mutton_path, 
             'poultry': poultry_path, 'pig': pig_path}
income_dict = {}

for group in income_groups: 
  ctrydf = categorydf[categorydf['Income group'].str.contains(group)]
  ctry_list = list(ctrydf.Economy) # gets list of countries within that income group
  
  
  dict_temp = {}
  for element in path_list:
    with open(path_list[element]) as csvfile: # opens the file associated with each meat
      csv_reader = csv.reader(csvfile, delimiter=',')
      header = next(csv_reader)
      header[0] = 'Country' # hard-coded!
      
      data_temp = [] # temporary storage for data
      for row in csv_reader: 
        if row[0] in ctry_list:
          data_temp.append(row) # add in that data
    
    groupdf = pd.DataFrame.from_records(data_temp, columns = header) # makes a dataframe of the csv
        # only for the countries in that income group
    slopes = list(groupdf['Slope'])
    slopes = [float(x) for x in slopes if x]
  
    pvals = list(groupdf['p'])
    pvals = [float(x) for x in pvals if x]

    #prop_pos = sum(1 for x in slopes if x > 0) / len(slopes)
    [prop_pos,prop_pos_sign] = slopes_counter(slopes,pvals)
    slope_mean = np.nanmean(slopes)
    
    dict_temp.update({element: {'pp': prop_pos, 'sig': prop_pos_sign, 'mean': slope_mean}})

  income_dict[group] = dict_temp
        
        
    

In [0]:
income_gdpmeat = pd.DataFrame.from_dict(income_dict)
income_gdpmeat.to_csv('gdrive/My Drive/Colab Notebooks/trends_by_incomepc2002.csv')


In [0]:
income_dict

# Food transformations analysis

In [0]:
# Creates a data frame from the data in 'path' and extracts the countries of interest
df = pd.read_csv(path_other)
df_gdp = pd.read_csv(path_gdp) # loads gdp data
country_list = unique(list(df['Country']))

In [0]:
unique(list(df['Item']))

['Poultry Meat', 'Milk - Excluding Butter']

In [0]:
# Runs the analysis for each commodity
commodity_list = unique(list(df['Item']))
for commodity in commodity_list:
  country_dict = {}
  for country in country_list: 
  
    country_ID = df['Country'].str.contains(country)
    country_data = df[country_ID] # this seems to grab only the "true" values from the above
    commodity_data = country_data[country_data['Item'].str.contains(commodity)]
    
    temp_dict = country_all_analysis(commodity_data, 'Food supply')
    country_dict[country] = temp_dict

    #print(country_dict)
    #break
  countrydf = pd.DataFrame.from_dict(country_dict, orient = 'index')

  savepath = 'gdrive/My Drive/Colab Notebooks/' + commodity + 'supp_ft.csv' # hopefully will always save 
      # to the correct commodity
  countrydf.to_csv(savepath)
  print([commodity, 'run'])
  
  

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  


['Poultry Meat', 'run']


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  


['Milk - Excluding Butter', 'run']


In [0]:
# runs the vs. GDP analysis
commodity = 'Milk - Excluding Butter'
gdp_type = 'Gross Domestic Product per capita'
country_dict = country_slope_calcs(commodity, gdp_type ,yearstart) # capitalization matters!
countrydf = pd.DataFrame.from_dict(country_dict, orient = 'index')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


In [0]:
# plots by country 
ctry_plot('Brazil',df,df_gdp,commodity)

NameError: ignored

# Fish data analysis

In [0]:
import csv

In [0]:
# load appropriate species names; need the 3 letter codes for each
with open (path_fish_IDS,'r') as csvfile:
  csv_reader = csv.reader(csvfile, delimiter=',')
  header = next(csv_reader)
  
  idx_family = header.index('Family') # grabs the location of the info we want in the header
  idx_code = header.index('3Alpha_Code')
  idx_name = header.index('Name_En')
  
  spp_header = ['Name', 'Code', 'Family']
  salmon_data = []
  cichlid_data = []
  Scode_only = []
  Ccode_only = []
  
  desired_family = ['SALMONIDAE', 'CICHLIDAE']
  for row in csv_reader:
    if (desired_family[0]) in row[idx_family]:
      salmon_data.append([row[idx_name], row[idx_code], row[idx_family]])
      Scode_only.append(row[idx_code])
    if (desired_family[1]) in row[idx_family]:
      cichlid_data.append([row[idx_name], row[idx_code], row[idx_family]])
      Ccode_only.append(row[idx_code])
# spp_data contains the codes for the species we're interested in
# spp_header has the header for this data
# idx_only just has the codes for ease of access


In [0]:
# loads the species we care about from the database
with open(path_fish,'r') as csv_file:
  csv_reader = csv.reader(csv_file, delimiter=',')
  header = next(csv_reader)
  
  idx_species = header.index('SPECIES')
  idx_country = header.index('COUNTRY')
  idx_yr = header.index('YEAR')
  idx_unit = header.index('QUANTITY_UNIT')
  idx_quant = header.index('QUANTITY')
  
  aqua_header = [header[idx_species], header[idx_country], header[idx_yr], header[idx_unit], header[idx_quant]]
  Saqua_data = []
  Caqua_data = []
  for row in csv_reader:
    if row[idx_species] in Scode_only:
      Saqua_data.append([row[idx_species], row[idx_country], row[idx_yr], row[idx_unit], row[idx_quant]])
    if row[idx_species] in Ccode_only:
      Caqua_data.append([row[idx_species], row[idx_country], row[idx_yr], row[idx_unit], row[idx_quant]])

In [0]:
  # identifies where in the header each data type occurs
  idx_speciesN = aqua_header.index('SPECIES')
  idx_countryN = aqua_header.index('COUNTRY')
  idx_yrN = aqua_header.index('YEAR')
  idx_unitN = aqua_header.index('QUANTITY_UNIT')
  idx_quantN = aqua_header.index('QUANTITY')
  

In [0]:
## Gets the name of each country associated with the numeric codes
with open(path_fish_country,'r') as csv_file:
  csv_reader = csv.reader(csv_file,delimiter=',')
  
  ctry_dict = {}
  
  for row in csv_reader:
    ctry_dict[row[0]] = row[4] # assigns name as the entry for key as number
  

In [0]:
# re-organize data into {country: {date: x, total amount of species in desired family: y}}
# Caqua_data_merge is the final dictionary for combined Cichlid species

Caqua_data_merge =  {} #{'0': {'years': [0], 'amounts': [0]}} # defines data structure

for row in Caqua_data: 

  if row[idx_countryN] in Caqua_data_merge: # if the country HAS appeared before
    # if the year has appeared before: (add species totals)
    if row[idx_yrN] in Caqua_data_merge[row[idx_countryN]]['years']:
      # find the index of the year we care about
      idx_yr = Caqua_data_merge[row[idx_countryN]]['years'].index(row[idx_yrN])
      yr_val = Caqua_data_merge[row[idx_countryN]]['amts'][idx_yr]
      Caqua_data_merge[row[idx_countryN]]['amts'][idx_yr] = yr_val + float(row[idx_quantN])

    # if the year hasn't appeared before: (new year)
    else:
      Caqua_data_merge[row[idx_countryN]]['years'].append(row[idx_yrN])
      Caqua_data_merge[row[idx_countryN]]['amts'].append(float(row[idx_quantN]))
      
      
  else: # add new row if the country hasn't appeared before

    ctry_name = ctry_dict[str(int(row[idx_countryN]))] # gets the associated country name
    #str(int()) necessary because digits less than 3 (e.g. 44) imported a 0 at the front

    temp_dict = {'years': [row[idx_yrN]], 'amts': [float(row[idx_quantN])], 'name': ctry_name}
    Caqua_data_merge[row[idx_countryN]] = temp_dict


In [0]:
# re-organize these data BACK into a csv type format for export
cichlid_export = []
for entry in Caqua_data_merge:
  total_length = len(Caqua_data_merge[entry]['amts']) # gets number of year / amts combos
  for i in range(0, total_length):
    row = [Caqua_data_merge[entry]['name'], Caqua_data_merge[entry]['years'][i],
          Caqua_data_merge[entry]['amts'][i]]
    cichlid_export.append(row)
  
cichliddf = pd.DataFrame(cichlid_export, columns = ['name', 'yr', 'amt'])
savepath = 'gdrive/My Drive/Colab Notebooks/' + 'cichlids' + 'all.csv' # hopefully will always save 
      # to the correct commodity
cichliddf.to_csv(savepath)

In [0]:
## By country analysis of 3 most common tilapia species: Oreochromis niloticus, mossambicus,
# and aureus

# find the codes of those species
spp_of_interest = ['Oreochromis niloticus', 'Oreochromis mossambicus', 'Oreochromis aureus']

with open (path_fish_IDS,'r') as csvfile:
  csv_reader = csv.reader(csvfile, delimiter=',')
  header = next(csv_reader)
  
  idx_spp = header.index('Scientific_Name') # grabs the location of the info we want in the header
  idx_code = header.index('3Alpha_Code')
  idx_name = header.index('Name_En')
 
  spp_dict = {}
  for row in csv_reader:
    for element in spp_of_interest:
      if element in row[idx_spp]:
        spp_dict[element] = row[idx_code]




SyntaxError: ignored

In [0]:
# loads the species we care about from the database; pulls codes from spp_dict
spp_codes = spp_dict.values()

with open(path_fish,'r') as csv_file:
  csv_reader = csv.reader(csv_file, delimiter=',')
  header = next(csv_reader)
  
  idx_species = header.index('SPECIES')
  idx_country = header.index('COUNTRY')
  idx_yr = header.index('YEAR')
  idx_unit = header.index('QUANTITY_UNIT')
  idx_quant = header.index('QUANTITY')
  
  aqua_header = ['Country', 'Year', spp_of_interest[0] + 'amt', 
                 spp_of_interest[1] + 'amt', spp_of_interest[2] + 'amt']
  Caqua_data = []
  for row in csv_reader:
    print(row[idx_species])
    break
    if row[idx_species] in spp_of_interest:
      Caqua_data.append([row[idx_species], row[idx_country], row[idx_yr], row[idx_unit], row[idx_quant]])


MZZ


In [0]:
spp_dict

{'Oreochromis aureus': 'OXW',
 'Oreochromis mossambicus': 'TLM',
 'Oreochromis niloticus': 'TLN'}

In [0]:
temp['158']['years'] = [1967, 1970]

In [0]:
temp

{'158': {'amts': 2027.0, 'years': [1967, 1970]}}

In [0]:
temp['158']['years']

[1967, 1970]

In [0]:
temp['158']['years'].index(1970)

1